# Brain Tumor Detection

### For Google Colab Users

In [ ]:
#@title ## Mount Your Google Drive & Map your Directory
#@markdown Please run this cell (`Ctrl+Enter` or `Shift+Enter`) and follow the steps printed bellow.
import os

from google.colab import drive
drive.mount('/content/gdrive')

# check Google Drive is mounted
if not os.path.isdir("/content/gdrive"):
  raise FileNotFoundError("Your Google Drive isn't mounted. Please run the above cell.")

assignment_dir = "/content/gdrive/MyDrive/workspace/BrainTumorDetection"  #@param{type:"string"}
assignment_dest = "/content/project"

# create symbolic link
!rm -f {assignment_dest}
!ln -s "{assignment_dir}" "{assignment_dest}"
print(f'Succesfully mapped (ln -s) "{assignment_dest}" -> "{assignment_dir}"')

# cd to linked dir
%cd -q {assignment_dest}
print(f'Succesfully changed directory (cd) to "{assignment_dest}"')

### Init

In [1]:
#@title ##Init

#@markdown Import required modules and observe your resources

# auto reload
%load_ext autoreload
%autoreload 2

# import
import torch
import torchvision
from PIL import Image
import matplotlib.pyplot as plt
import random
import os
import wandb
wandb.login()

from experiment import Experiment
from post_process import get_masks_from_heatmaps

# Show resources
!nvidia-smi

wandb: Currently logged in as: sgvdan (use `wandb login --relogin` to force relogin)


Sat Jul 10 17:27:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 6000     Off  | 00000000:15:00.0 Off |                  Off |
| 33%   29C    P8    16W / 260W |      0MiB / 24220MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Experiment

In [ ]:
# params
config = {
"model_name": "vgg19",
#"model_name": "vgg11",
"data_train_path": os.path.join(".", "Data", "Training"),
"data_test_path": os.path.join(".", "Data", "Testing"),
"epochs": 6,
"train_batch_size": 200,
"lr": 1e-4,
"heatmap_batch_size": 256,
"occlusion_size": (60,60),
"heat_layers" : [['features', layer_num] for layer_num in [0, 2, 5, 7, 10, 12, 14, 16, 19, 21, 23, 25, 28, 30, 32, 34]] +
                [['classifier', -1]],
"heatmap_threshold" : 0.80,
"shuffle_data": True,
"device": "cuda"
}

with wandb.init(project="BrainTumorDetection", config=config):
    # load best model
    experiment = Experiment(config)
#     experiment.train_model()
    print("Model's accuracy: ", "{:.2f}".format(experiment.eval_model()), flush=True)
    
    # generate heatmaps
    heatmaps = experiment.generate_heatmap(r"./Data/Testing/glioma_tumor/image(3).jpg")
    
    # calculate masks
    hot_masks, cold_masks = get_masks_from_heatmaps(heatmaps, thresh=config["heatmap_threshold"], smallest_contour_len=30)

loading vgg19 model...
loading best model state from: ./Models/vgg19.pth
Model's accuracy:  0.98
Creating heatmap...


 20%|█▉        | 50/256 [00:33<02:29,  1.38it/s]

In [ ]:
from matplotlib import pyplot as plt
plt.figure(); plt.imshow(occluded_images[100,0,:,:].cpu(), cmap="gray"); plt.show()